##### Name: K Lalith Aditya
##### Regd No: 22231
##### Building the MODEL

<h4>BUILD the neural Network</h4>
<br>
Neural networks comprise of layers/models that perform operations on the data. The torch.nn provides all the building blocks you need to build your own neural network. Every module in pytorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). The nested structure allows for building and managing complex architectures easily.

In [1]:
# Building the Neural Network to Classify the images in the FashionMNIST dataset

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
## Checking whether there is a hardware accelerator like the GPU or MPS. 
## or torch.cuda or torch.backends.mps are available or we use CPU.

device = (
    "cuda"
    if torch.cuda.is_available() # checking for cuda
    else "mps"
    if torch.backends.mps.is_available() # checking for mps
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
# Define the Class

"""
We define our neural network by subclassing nn.Module and initialize the neural network layers in __init__. Every 
nn.Module subclass implemetns the operatios on the input data in the forward method.
"""

'\nWe define our neural network by subclassing nn.Module and initialize the neural network layers in __init__. Every \nnn.Module subclass implemetns the operatios on the input data in the forward method.\n'

In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self): # This is the initialization method of the NeuralNetwork class. It is executed when an instance of the class is created.
        super().__init__() # calls the initialization method of the parent class 
        self.flatten = nn.Flatten() #creates an instance of the nn.Flatten module. This module is used to flatten the input tensor, which is typically a multi-dimensional tensor, into a 1-dimensional tensor. This is required before passing the input through the fully connected layers.
        self.linear_relu_stack = nn.Sequential(
            # defines a sequential container module called linear_relu_stack. It consists of a sequence of linear (fully connected) layers followed by ReLU activation functions.
            nn.Linear(28*28, 512), # This is the first linear layer with 28*28 (784) input features and 512 output features.
            nn.ReLU(), # This applies the Rectified Linear Unit (ReLU) activation function element-wise to the output of the first linear layer.
            nn.Linear(512,512), #  This is the second linear layer with 512 input features and 512 output features.
            nn.ReLU(), # This applies ReLU activation to the output of the second linear layer.
            nn.Linear(512,10),
            # This is the final linear layer with 512 input features (output from the previous layer) and 10 output features, representing the logits for the 10 classes.
        )
        
    def forward(self,x):
        x = self.flatten(x) #  applies the flatten module to flatten the input tensor x.
        logits = self.linear_relu_stack(x) #passes the flattened input through the linear_relu_stack module, which applies the linear layers and ReLU activations in sequence.
        return logits # return the logits value
        

In [22]:
## Creating an instance of NeuralNetwork, and move it to the device and print its structure

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [23]:
"""
To use the model, we pass the input data. This executes the model's forward, along with some
background operations. Do not call the model.forward() directly

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw
predicted values for each class. and dim=1 corresponding to the individual values of each output.
We get the prediction probabilities by passing it through the instance of the nn.Softmax module
"""

"\nTo use the model, we pass the input data. This executes the model's forward, along with some\nbackground operations. Do not call the model.forward() directly\n\nCalling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw\npredicted values for each class. and dim=1 corresponding to the individual values of each output.\nWe get the prediction probabilities by passing it through the instance of the nn.Softmax module\n"

In [35]:
X = torch.rand(1, 28, 28, device=device)
# This line creates a random tensor X with shape (1, 28, 28) on the specified device. It represents the input data, possibly an image, for the neural network.
logits = model(X)
# The model performs the forward pass, applying its layers to generate the logits. Logits are the raw output values from the neural network before applying any activation function.
pred_probab = nn.Softmax(dim=1)(logits)
# This line applies the softmax function to the logits using nn.Softmax. The softmax function converts the logits into probabilities, where each value represents the predicted probability of the corresponding class label. The dim=1 argument indicates that the softmax operation should be performed along dimension 1 (the class dimension).
y_pred = pred_probab.argmax(1)
# This line uses the argmax function to find the index of the class label with the highest probability. It returns a tensor y_pred containing the predicted class label for the input data.
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


In [38]:
# Model layers
"""
Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.
"""
input_image = torch.rand(3,28,28) # 3 images of size 28*28
print(input_image.size())

torch.Size([3, 28, 28])


In [41]:
# nn.Flatten
# We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).
flatten = nn.Flatten() # creates an instance and assigns to flatten 
flat_image = flatten(input_image) # flattening the image
print(flat_image.size())

torch.Size([3, 784])


In [42]:
# nn.Linear
# The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [43]:
## nn.ReLU
# Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

#In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.

print(f"Before ReLU: {hidden1}\n\n") #before ReLU activation
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}") #after ReLU activation

Before ReLU: tensor([[ 0.2439, -0.1054, -0.1111,  0.3703,  0.4812, -0.1558,  0.5489,  0.7603,
         -0.4405,  0.3205, -0.4583,  0.0818, -0.2027,  0.3265, -0.0573,  0.4570,
         -0.1445,  0.3469,  0.3093, -0.5367],
        [-0.0210, -0.0928, -0.1637,  0.1365,  0.3346, -0.2265,  0.7167,  0.3921,
         -0.0046,  0.0700, -0.7507,  0.4989,  0.2835,  0.3052, -0.3222,  0.1745,
         -0.0856,  0.2226,  0.1520, -0.7739],
        [ 0.1493, -0.1463,  0.2821,  0.1663,  0.3590, -0.0069,  0.4745,  0.5459,
         -0.1242,  0.1097, -0.8123, -0.0345, -0.0184,  0.3922, -0.2342,  0.3006,
         -0.3071,  0.3864, -0.0104, -0.7499]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2439, 0.0000, 0.0000, 0.3703, 0.4812, 0.0000, 0.5489, 0.7603, 0.0000,
         0.3205, 0.0000, 0.0818, 0.0000, 0.3265, 0.0000, 0.4570, 0.0000, 0.3469,
         0.3093, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.1365, 0.3346, 0.0000, 0.7167, 0.3921, 0.0000,
         0.0700, 0.0000, 0.4989, 0.2835, 0.3052, 0.00

In [49]:
# nn.Sequential
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
) # sequence of the layers
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [50]:
# nn.Softmax

"""
The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.
"""

'\nThe last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.\n'

In [51]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

#### Model Parameters

In [53]:
#Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In [54]:
print(f"Model structure: {model}\n\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)




In [56]:
# iterating over the parameters
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n") 

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0149, -0.0310,  0.0055,  ...,  0.0307,  0.0297,  0.0212],
        [-0.0058,  0.0013,  0.0310,  ..., -0.0013, -0.0132, -0.0187]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0134, -0.0087], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0048,  0.0073,  0.0188,  ..., -0.0159, -0.0349, -0.0076],
        [ 0.0288,  0.0200,  0.0244,  ..., -0.0347,  0.0252, -0.0208]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([-0.0252, -0.0397], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0351,  0.0157, -0.0196,  ..., -0.0307,  0.0091,  0.0031],
        [-0.0389, -0.0338,  0.0164,  ...,  0.0068,  0.0256,  0.0136]],
       grad_fn=<SliceBackward0>)